This notebook is freely available for redistribution under the [GPL-3.0 license](https://choosealicense.com/licenses/gpl-3.0/).

Author: 蘇嘉冠

Contributors: 鄭宇伸, 喬彥翔

# Multilayer Perceptron 與影像分類（參考答案）

請記得先複製一份在你自己的 Google 帳號底下：
`檔案` -> `在雲端硬碟中儲存副本`

也請記得改變 colab 設定：
1. `工具` -> `設定` -> `編輯器` -> 將 `縮排寬度` 改為 `4`
2. `編輯` -> `筆記本設定` -> 將 `硬體加速器` 改為 `GPU`

## 練習題

MNIST 為深度學習最常用來入門的資料集，可以說是深度學習的 Hello World。這個資料集主要是由 Yann LeCun 在 1998 所蒐集的（Yann LeCun 為深度學習最重要的貢獻者，也是 Convolutional Neural Networks 的發明人），蒐集數字 0 ~ 9 的手寫數字圖片（分別對應的 label 為 `0` ~ `9`），每張圖片大小為 28 x 28。

這次練習，我們將用 PyTorch 來建構一個最簡單的 MLP，並且用 MNIST 做訓練，來預測手寫圖片為何種數字。

In [ ]:
!pip install numpy matplotlib torch torchvision

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch

### Data Preprocessing

我們要準備 MNIST 的資料，準備的方式通常有兩種：
1. 自己下載 MNIST 資料到 colab 的電腦中，再做讀取
2. 直接使用 [torchvision.datasets](https://pytorch.org/vision/stable/datasets.html) 下載 MNIST 資料，再做讀取

這次練習使用方法 2：
- 透過 `torchvision.datasets` 下載 MNIST 後，並且指定將資料轉成 [Tensor](https://pytorch.org/tutorials/beginner/blitz/tensor_tutorial.html#sphx-glr-beginner-blitz-tensor-tutorial-py) 型別
- 創建 [DataLoader](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html) 的物件，這個物件負責實際的讀取資料，而且能指定 batch size 的大小（也就是一次要讀幾張圖片）

In [ ]:
import torchvision.transforms as transforms
from torchvision import datasets

# Batch size: how many samples per batch to load.
batch_size = 16

# Convert data to torch Tensor.
transform = transforms.ToTensor()

# Download the training and testing data for MNIST.
train_data = datasets.MNIST(
    "mnist_data",
    train=True,
    download=True,
    transform=transform,
)
test_data = datasets.MNIST(
    "mnist_data",
    train=False,
    download=True,
    transform=transform,
)

# Create loaders for training and testing data.
train_loader = torch.utils.data.DataLoader(
    train_data,
    shuffle=True,
    batch_size=batch_size,
)
test_loader = torch.utils.data.DataLoader(
    test_data,
    shuffle=True,
    batch_size=batch_size,
)

創建好 `DataLoader` 物件後，我們可以從中抓一把資料，一把資料的資料數量就是 batch size 的大小，每筆包含圖片與相對應的 label，圖片與 label 的型別都是 `Tensor`。由於我們在創建 `DataLoader` 物件時有指定 `shuffle=True`，所以每次抓的資料都會不一樣。

可以看到圖片的 shape 為 `[16, 1, 28, 28]`，這種格式稱之為 `NCHW`，分別代表的意義如下：
- `N`：batch size 大小
- `C`：圖片的 channel 數量，灰階圖為 1，彩色圖通常為 3
- `H`：圖片的高度（height）
- `W`：圖片的寬度（width）

另外，`Tensor` 型別可以與 `numpy.ndarray` 互相轉換型別。

In [ ]:
# Obtain one batch of training images.
images, labels = iter(train_loader).next()

print("Images shape: {}".format(images.shape))
print("Labels shape: {}".format(labels.shape))
print("Images: {}".format(images))
print("Labels: {}".format(labels))

# We can also transform from Tensor to numpy.ndarray
images = images.numpy()
print(images.shape)

### Data Visualization

同樣的，我們可以將從 `DataLoader` 物件拿出來的資料做視覺化。

In [ ]:
# Obtain one batch of training images.
images, labels = iter(train_loader).next()
images = images.numpy()

# Plot the images in the batch, along with the corresponding labels.
fig = plt.figure(figsize=(25, 4))
for idx in np.arange(batch_size):
    ax = fig.add_subplot(2, batch_size / 2, idx + 1, xticks=[], yticks=[])
    ax.imshow(np.squeeze(images[idx]), cmap='gray')
    # Print out the correct label for each image
    # .item() gets the value contained in a Tensor
    ax.set_title(str(labels[idx].item()))

    fig.show()

### Training

開始訓練之前，我們要先定義 Neural Network 的架構。[torch.nn](https://pytorch.org/docs/stable/nn.html) 包含許多深度學習的模組，例如我們這裡會用到的 `nn.Linear`（Fully-Connected layer）、`nn.Sigmoid`，透過這些模組的串連來組成我們想要的模型（下圖）。

要注意的是，由於我們等等會使用的 Cross Entropy 來當作 loss function，而 PyTorch 的 Cross Entropy 功能本身包含 Softmax，所以不會在這裡用到，而是等到訓練後的 inference 再加上去。

定義模型架構之後，因為我們想要用 GPU 加速，因此使用 `model.cuda()` 將模型的相關資料送到 GPU。如果只想在 CPU 上執行，可以將此行註解掉。

![](https://i.imgur.com/8n9LzOD.png)

In [ ]:
import torch.nn as nn

# Construct the neural network.
model = nn.Sequential(
    nn.Linear(28 * 28, 512),
    nn.ReLU(),
    nn.Linear(512, 64),
    nn.ReLU(),
    nn.Linear(64, 10),
)
# Move model from CPU to GPU.
model = model.cuda()
print(model)

我們也要指定要用到的 loss function 以及優化器（更新權重的方法），分別使用 Cross Entropy 以及 SGD。

In [ ]:
# Specify loss function.
criterion = nn.CrossEntropyLoss()

# Specify optimizer.
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

接下來正式做訓練的流程，基本上流程與 Lecture 4 投影片講述的一樣。

In [ ]:
# Number of epochs to train the model.
n_epochs = 30

# Turn on training mode.
model.train()

for epoch in range(n_epochs):
    # Monitor training loss.
    train_loss = 0.0
    
    # Train the model.
    for data, target in train_loader:
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # Reshape data from (16, 1, 28, 28) into (16, 28 * 28).
        data = data.view(-1, 28 * 28)
        # Move data from CPU to GPU.
        data = data.cuda()
        target = target.cuda()
        # Forward pass:
        # Compute predicted outputs by passing inputs to the model.
        output = model(data)
        # Calculate the loss.
        loss = criterion(output, target)
        # Backward pass:
        # Compute gradient of the loss with respect to model parameters.
        loss.backward()
        # Perform a single optimization step (parameter update).
        optimizer.step()
        # Update running training loss.
        train_loss += loss.item() * data.size(0)
        
    # Calculate average loss over an epoch.
    train_loss = train_loss / len(train_loader.dataset)

    print('Epoch: {} \tTraining Loss: {}'.format(
        epoch + 1, 
        train_loss
    ))

### Evaluation

訓練完成後，我們可以用模型來做 inference。這裡示範從 testing data 抓一把資料，餵給模型來做預測。要注意的是由於我們定義的模型並沒有包含 Softmax，所以在 inference 的時候我們手動加上去，這樣預測的結果就會是每個 label 的機率，我們就可以將機率做大的 label 作為預測的 label。

In [ ]:
import torch.nn.functional as F

# Turn off training mode (i.e., turn on evaluation mode).
model.eval()

# Obtain one batch of testing images.
images, labels = iter(test_loader).next()

# Get sample outputs.
input_images = images.view(-1, 28 * 28)
input_images = input_images.cuda()
output = model(input_images)
output = F.softmax(output)
print(output.shape)

# Convert output probabilities to predicted class.
_, preds = torch.max(output, 1)

# Prepare images for display.
images = images.numpy()

# Plot the images in the batch, along with predicted and true labels.
fig = plt.figure(figsize=(25, 4))
for idx in np.arange(batch_size):
    ax = fig.add_subplot(2, batch_size / 2, idx + 1, xticks=[], yticks=[])
    ax.imshow(np.squeeze(images[idx]), cmap='gray')

    title = "{} ({})".format(preds[idx].item(), labels[idx].item())
    color = "green" if preds[idx] == labels[idx] else "red"
    ax.set_title(title, color=color)

接下來我們可以用全部的 testing data 來算準確率，分別計算各個 label 的準確率，以及全部之下的準確率。

In [ ]:
class_correct = [0] * 10
class_total = [0] * 10

for data, target in test_loader:
    # Get data outputs.
    data = data.view(-1, 28 * 28)
    data = data.cuda()
    target = target.cuda()
    output = model(data)
    output = F.softmax(output)
    # Convert output probabilities to predicted class.
    _, pred = torch.max(output, 1)
    # Compare predictions to true label.
    correct = np.squeeze(pred.eq(target.data.view_as(pred)))
    # calculate test accuracy for each object class
    for i in range(batch_size):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1

for label in range(10):
    print("Test Accuracy of {}: {}% ({} / {})".format(
        label,
        100 * class_correct[label] / class_total[label],
        int(np.sum(class_correct[label])),
        int(np.sum(class_total[label]))),
    )
print("Test Accuracy (Overall): {}% ({} / {})".format(
    100 * np.sum(class_correct) / np.sum(class_total),
    int(np.sum(class_correct)),
    int(np.sum(class_total))),
)